In [14]:
from rpy2 import robjects
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import STAP
import pandas as pd
utils = importr('utils')
utils.install_packages('SentimentAnalysis')
import rpy2.rlike.container as rlc
from rpy2.robjects import r, pandas2ri

In [2]:
headlinedata = pd.read_csv('abcnews-date-text.csv')
headlines = headlinedata['headline_text']

In [3]:
# File path to R file
pathSentimentR = "sentimentR.R"

In [4]:
# Transfer data to an R string vector
headlineVector = robjects.StrVector(headlines)

In [5]:
# Function that calls the R functions and outputs the sentiments value (numerical)
def getSentiments(headline):
    with open(pathSentimentR, 'r') as f:
        string = f.read()
    sentimentR = STAP(string, "sentimentR")
    sentiment_value = sentimentR.sent(headline)
    return sentiment_value

In [31]:
# Create pandas dataframe to hold sentiment values
pandas2ri.activate()
columns = ['WordCount', 'SentimentGI', 'NegativityGI', 'PositivityGI', 'SentimentHE',
          'NegativityHE', 'PositivityHE', 'SentimentLM', 'NegativityLM',
          'PositivityLM', 'RatioUncertaintyLM', 'SentimentQDAP', 'NegativityQDAP', 
           'PositivityQDAP']
sentiment_scores = pd.DataFrame(columns=columns)

In [34]:
# Example loop to show usage
for i in range(0,3):
    score = getSentiments(headlineVector[i])
    sentiment_scores = sentiment_scores.append(pandas2ri.ri2py(score))

In [35]:
# Transferring to dataframe shows results: Example below of numbers in output
#   WordCount SentimentGI NegativityGI PositivityGI SentimentHE
#         5         0.2            0          0.2           0
#  NegativityHE PositivityHE SentimentLM NegativityLM
#            0            0           0            0
#  PositivityLM RatioUncertaintyLM SentimentQDAP
#            0                  0             0
#  NegativityQDAP PositivityQDAP
#              0              0
# Initial analysis was done using SentimentQDAP score
sentiment_scores.head()

,WordCount,SentimentGI,NegativityGI,PositivityGI,SentimentHE,NegativityHE,PositivityHE,SentimentLM,NegativityLM,PositivityLM,RatioUncertaintyLM,SentimentQDAP,NegativityQDAP,PositivityQDAP
1,5.0,0.200000,0.000000,0.200000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00
1,6.0,-0.166667,0.333333,0.166667,0.0,0.0,0.0,-0.333333,0.333333,0.0,0.0,-0.166667,0.166667,0.00
1,4.0,0.750000,0.000000,0.750000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.250000,0.000000,0.25
